---
title: "Luigi Mangione Extradition Probability"
date: last-modified
author: Your Name Here
format:
  html:
    echo: false
    embed-resources: true
    include-in-header:
      - text: |
          <script src="https://cdnjs.cloudflare.com/ajax/libs/highcharts/11.2.0/highcharts.js"></script>
          <script src="https://cdnjs.cloudflare.com/ajax/libs/highcharts/11.2.0/modules/exporting.js"></script>
          <script src="https://cdnjs.cloudflare.com/ajax/libs/highcharts/11.2.0/modules/export-data.js"></script>
          <script src="https://cdnjs.cloudflare.com/ajax/libs/highcharts/11.2.0/modules/accessibility.js"></script>
execute:
  allow: true
---

In [ ]:
#| echo: false
#| output: asis

import polars as pl
import json
from datetime import datetime

def process_market_data(df):
    series_data = []
    
    # Get unique end dates
    end_dates = df['end_date'].unique().to_list()
    
    for end_date in end_dates:
        df_market = df.filter(pl.col('end_date') == end_date)
        
        # Convert timestamp to milliseconds and prepare trading data
        trading_data = (
            df_market.select([
                (pl.col('full_timestamp').dt.timestamp().mul(1000)).alias('x'),
                pl.col('price').alias('y')
            ])
            .with_columns([
                pl.col('x').cast(pl.Int64),
                pl.col('y').cast(pl.Float64)
            ])
            .to_numpy()
            .tolist()
        )
        
        # Instead of just min and max timestamps, use all timestamps
        # but with the same yes_quote value for each timestamp
        yes_quote = float(df_market['yes_quote'][0])
        yes_quote_data = [
            [x, yes_quote] for x, _ in trading_data
        ]
        
        # Format end date for display
        display_date = end_date.strftime("%Y-%m-%d") if isinstance(end_date, datetime) else str(end_date)
        
        # Add data series
        series_data.extend([
            {
                'name': f'Trading Price (End: {display_date})',
                'data': trading_data
            },
            {
                'name': f'Current Yes Quote ({display_date})',
                'data': yes_quote_data
            }
        ])
    
    return series_data

# Process data
df = pl.read_parquet('luigi-mangione-extradited-to-new-york-in_2024-12-15.parquet')
chart_data = process_market_data(df)

# Output as JavaScript
print(f"<script>const chartData = {json.dumps(chart_data)};</script>")

```{=html}
<div class="chart-container">
    <div id="chart"></div>
</div>

<style>
.chart-container {
    background: white;
    width: 100%;  /* Full width */
    margin: 20px auto;
    padding: 20px;
}
#chart {
    width: 100%;
    height: 600px;
    min-width: 800px;  /* Minimum width */
}
</style>

<script>
document.addEventListener('DOMContentLoaded', function() {
    // Process the data and add visual styling properties
    const processedData = chartData.map(series => {
        const isQuoteLine = series.name.includes('Quote');
        return {
            name: series.name,
            data: series.data.map(point => {
                let timestamp = Number(point[0].toString());
                if (timestamp > 1e16) timestamp = timestamp / 1000000;
                else if (timestamp > 1e13) timestamp = timestamp / 1000;
                return [timestamp, point[1]];
            }),
            // Add specific styling for quote lines
            opacity: isQuoteLine ? 0.3 : 1,
            lineWidth: isQuoteLine ? 1 : 2,
            dashStyle: isQuoteLine ? 'Dash' : 'Solid',
            enableMouseTracking: true // Keep tooltip functionality
        };
    });

    Highcharts.chart('chart', {
        chart: {
            type: 'line',
            style: {
                fontFamily: '-apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", Roboto, "Helvetica Neue", Arial, sans-serif'
            }
        },
        
        title: {
            text: 'Luigi Mangione Extradition Probability',
            align: 'left',
            style: {
                fontSize: '20px',
                fontWeight: 'bold'
            }
        },

        subtitle: {
            text: 'Betting market probabilities by expiry date',
            align: 'left'
        },

        xAxis: {
            type: 'datetime',
            dateTimeLabelFormats: {
                millisecond: '%b %e',
                second: '%b %e',
                minute: '%b %e',
                hour: '%b %e',
                day: '%b %e',
                week: '%b %e',
                month: '%b',
                year: '%Y'
            },
            labels: {
                rotation: -45,
                align: 'right',
                style: {
                    fontFamily: 'Inter, JetBrains Mono, monospace',
                    fontSize: '12px',
                    color: '#666'
                }
            }
        },
        
        yAxis: {
            title: { text: null },
            min: 0,
            max: 1,
            tickInterval: 0.2,
            labels: { 
                formatter: function() {
                    return Math.round(this.value * 100) + '%';
                },
                style: {
                    color: '#666666',
                    fontSize: '14px',
                    fontFamily: 'Inter, sans-serif'
                }
            },
            gridLineWidth: 1,
            gridLineDashStyle: 'Dot',
            gridLineColor: '#E0E0E0'
        },
        
        tooltip: {
            shared: true,
            useHTML: true,
            formatter: function() {
                if (!this.points) return false;
                
                const date = new Date(this.x);
                const formattedDate = date.toLocaleDateString('en-GB', {
                    year: 'numeric',
                    month: 'long',
                    day: 'numeric',
                    hour: '2-digit',
                    minute: '2-digit'
                });
                
                const expiryGroups = {};
                
                this.points.forEach(point => {
                    const seriesName = point.series.name;
                    let expiryDate = '';
                    
                    if (seriesName.includes('End:')) {
                        expiryDate = seriesName.split('End: ')[1].replace(')', '');
                    } else if (seriesName.includes('Quote')) {
                        expiryDate = seriesName.match(/\((.*?)\)/)[1];
                    }
                    
                    if (!expiryGroups[expiryDate]) {
                        expiryGroups[expiryDate] = {
                            trading: null,
                            quote: null
                        };
                    }
                    
                    if (seriesName.includes('Trading Price')) {
                        expiryGroups[expiryDate].trading = point.y;
                    } else if (seriesName.includes('Quote')) {
                        expiryGroups[expiryDate].quote = point.y;
                    }
                });
                
                let html = `<div style="font-family: Inter; padding: 8px;">
                    <div style="font-weight: bold; margin-bottom: 8px;">${formattedDate}</div>`;
                
                Object.entries(expiryGroups).forEach(([expiryDate, values]) => {
                    html += `<div style="margin-top: 8px;">
                        <div style="font-weight: bold; color: #666;">Expiry: ${expiryDate}</div>`;
                    
                    if (values.trading !== null) {
                        html += `<div style="color: #2f7ed8; margin: 2px 0;">
                            Trading: ${(values.trading * 100).toFixed(1)}%</div>`;
                    }
                    
                    if (values.quote !== null) {
                        html += `<div style="color: #0d233a; margin: 2px 0;">
                            Quote: ${(values.quote * 100).toFixed(1)}%</div>`;
                    }
                    
                    if (values.trading !== null && values.quote !== null) {
                        const diff = values.trading - values.quote;
                        const diffColor = diff > 0 ? '#059669' : '#EC4899';
                        html += `<div style="color: ${diffColor}; margin: 2px 0;">
                            Difference: ${(diff > 0 ? '+' : '')}${(diff * 100).toFixed(1)}%</div>`;
                    }
                    
                    html += '</div>';
                });
                
                html += '</div>';
                return html;
            }
        },
        
        plotOptions: {
            series: {
                marker: {
                    enabled: false,
                    states: {
                        hover: {
                            enabled: true,
                            radius: 4
                        }
                    }
                },
                states: {
                    hover: {
                        enabled: true,
                        lineWidth: 4
                    }
                }
            }
        },
        
        legend: {
            align: 'center',
            verticalAlign: 'bottom',
            layout: 'horizontal',
            itemStyle: {
                color: '#666',
                fontWeight: '500',
                fontFamily: 'Inter, sans-serif'
            },
        },
        
        series: processedData,
        
        credits: {
            enabled: false
        }
    });
});
</script>


```